In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import Image, display
import glob
import tqdm

# ============================================
# 1. Dynamics definitions
# ============================================

def deriv(x, y, px, py, m=1.0, q=1.0, Phi0=0.01):
    """Compute derivatives for the Hamiltonian system."""
    dx = px / m
    dy = py / m
    dpx = 2.0 * np.pi * q * Phi0 * np.sin(2.0 * np.pi * x)
    dpy = 2.0 * np.pi * q * Phi0 * np.sin(2.0 * np.pi * y)
    return dx, dy, dpx, dpy


def integrate_single_particle(x0, y0, px0, py0, T=80.0, dt=0.0025):
    """Integrate a single particle trajectory using Leapfrog with periodic BC."""
    steps = int(T / dt)
    x, y, px, py = x0, y0, px0, py0
    x_hist = np.zeros(steps + 1)
    y_hist = np.zeros(steps + 1)
    x_hist[0], y_hist[0] = x, y

# Correct Leapfrog stepper (only wrap positions AFTER the full Leapfrog update)

# tqdm for progress bar
    for n in tqdm.tqdm(range(steps), desc="Integrating particle"):
        # compute acceleration/force and momentum kick
        _, _, kpx, kpy = deriv(x, y, px, py)

        # half kick first
        px += 0.5 * dt * kpx
        py += 0.5 * dt * kpy

        # drift using updated momentum
        kx, ky, _, _ = deriv(x, y, px, py)
        x += dt * kx
        y += dt * ky
        
        # then another half kick
        _, _, kpx, kpy = deriv(x, y, px, py)
        px += 0.5 * dt * kpx
        py += 0.5 * dt * kpy

        x %= 1.0
        y %= 1.0

        x_hist[n + 1] = x
        y_hist[n + 1] = y

    return x_hist, y_hist


# ============================================
# 2. Plotting
# ============================================

def plot_combined_trajectories(pattern="Lorentz_Part_*.csv",
                               sim_x=None, sim_y=None,
                               out_png="combined_trajectories.png"):
    """Plot Lorentz CSV trajectories and a simulated one together."""

    files = sorted(glob.glob(pattern))
    if not files:
        raise FileNotFoundError(f"No files matching pattern: {pattern}")

    # Read Lorentz data
    frames = []
    for i, f in enumerate(files):
        df = pd.read_csv(f)
        required = {"id", "rank", "X", "Y"}
        if not required.issubset(df.columns):
            raise ValueError(f"File {f} missing required columns: {list(df.columns)}")
        df["frame"] = i
        frames.append(df.loc[:, ["id", "rank", "X", "Y", "frame"]])
    data = pd.concat(frames, ignore_index=True)

    # Determine axis limits
    xlim = [0, 1]
    ylim = [0, 1]

    # Plot
    fig, ax = plt.subplots(figsize=(6, 6))
    # Lorentz trajectories
    for pid, g in data.groupby("id", sort=False):
        g = g.sort_values("frame")
        ax.plot(g["X"], g["Y"], label="mfem")

    # Simulated trajectory (overlay)
    if sim_x is not None and sim_y is not None:
        ax.plot(sim_x, sim_y, label="Python")

    # Plot starting points
    start_points = (sim_x[0], sim_y[0])
    ax.scatter(start_points[0], start_points[1], color="red", s=20, label="Start Points")
    # Plot ending points
    end_points = (sim_x[-1], sim_y[-1])
    ax.scatter(end_points[0], end_points[1], color="green", s=20, label="End Points")

    # ax.set_xlim(*xlim)
    # ax.set_ylim(*ylim)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_aspect("equal")
    ax.set_title(f"Combined Lorentz and Simulated Trajectories ({len(files)} frames)")
    ax.legend(loc="upper right", fontsize="small", markerscale=0.7)

    plt.savefig(out_png, dpi=150, bbox_inches="tight")
    plt.close(fig)
    display(Image(filename=out_png))
    print(f"Saved combined figure to: {out_png}")


# ============================================
# 3. Main Execution
# ============================================

if __name__ == "__main__":
    # Simulate a single particle
    x0, y0 = 0.1, 0.1
    px0, py0 = 0.01, 0.02
    sim_x, sim_y = integrate_single_particle(x0, y0, px0, py0, T=200, dt=0.005)

    # Combine plot
    plot_combined_trajectories(pattern="Lorentz_Part_*.csv",
                               sim_x=sim_x, sim_y=sim_y,
                               out_png="combined_trajectories.png")